In [70]:
import csv
import psycopg2
import requests
from datetime import date

# PSQL 
Install postgres by running 
```bash
 sudo apt-get update
 sudo apt-get install postgresql postgresql-contrib
```
Edit the file /etc/postgresql/<version>/main/pg_hba.conf and find the following line:
```
local   all             postgres                  md5
change it to:
local   all             postgres                 trust
 ```
Edith postgresql.conf and and listen_addresses = '*
```bash
sudo nano /etc/postgresql/9.5/main/postgresql.conf
listen_addresses = '*' 
sudo service postgresql reload
```
After that reload postgres by running
```bash
sudo service postgresql reload
```
## Login to postgres and create user and database
Postgres installation created a user account called postgres that is associated with the default Postgres role. In order to use Postgres, we can log into that account.This can be done as shown below:
```bash
sudo -i -u postgres
```
You can now access a Postgres prompt immediately by typing:
```bash
psql
```
you can exit the interactive Postgres session by typing:
```bash
postgress=# \q
```
## Create a New User and database
you can create a new user by typing:
```sql
CREATE USER denco WITH PASSWORD 'somepassword' NOSUPERUSER
CREATE DATABASE denco OWNER denco
```
## View Database and Tables in postgres
To view database type:
```bash
postgres=# \l
```
To connect to a database:
```bash
postgres=# \c denco
```
To list tables in a database:
``` bash
postgres=# \l 
```

In [72]:
conn = psycopg2.connect("dbname=dennis user=dennis host=127.0.0.1 password=dennis")
cur = conn.cursor()
cur.execute("""
    CREATE TABLE ign_reviews (
        id BIGINT PRIMARY KEY,
        score_phrase VARCHAR(11),
        title TEXT,
        url TEXT,
        platform VARCHAR(20),
        score DECIMAL(3, 1),
        genre TEXT,
        editors_choice BOOLEAN,
        release_date DATE
    )
""")

In [73]:
res= requests.get("https://dsserver-prod-resources-1.s3.amazonaws.com/246/ign.csv")
decoded_res = res.content.decode('utf-8')
reader = csv.reader(decoded_res.splitlines(), delimiter=',')
next(reader)
data_list = list(reader)

In [80]:
data_list[0]

['5249979066121302517',
 'Amazing',
 'LittleBigPlanet PS Vita',
 '/games/littlebigplanet-vita/vita-98907',
 'PlayStation Vita',
 '9.0',
 'Platformer',
 'Y',
 '2012',
 '9',
 '12']

In [75]:
for row in data_list:
    updated_row = row[:8]
    updated_row.append(date(int(row[8]), int(row[9]), int(row[10])))
    cur.execute("INSERT INTO ign_reviews VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)", updated_row)
conn.commit()

In [78]:
updated_row

['5012610255156399104',
 'Masterpiece',
 'Inside',
 '/games/inside-playdead/pc-20055740',
 'PC',
 '10.0',
 'Adventure',
 'Y',
 datetime.date(2016, 6, 28)]

In [65]:
cur.execute("SELECT * FROM ign_reviews")
data=cur.fetchall()

In [81]:
data[0]

(5249979066121302517,
 'Amazing',
 'LittleBigPlanet PS Vita',
 '/games/littlebigplanet-vita/vita-98907',
 'PlayStation Vita',
 Decimal('9.0'),
 'Platformer',
 True,
 datetime.date(2012, 9, 12))